In [14]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma

In [15]:
from dotenv import load_dotenv

load_dotenv()

False

In [16]:
import pandas as pd

games = pd.read_csv("games_cleaned.csv",index_col="appid")

In [17]:
games

,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,header_image,...,median_playtime_2weeks,discount,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent,tags_text,tagged_description
appid,,,,,,,,,,,,,,,,,,,,,
730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,https://shared.akamai.steamstatic.com/store_it...,...,350,0,1212356,"{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...",86,8632939,82,96473,FPS Shooter Multiplayer Competitive Action Tea...,"For over two decades, Counter-Strike has offer..."
578080,PUBG: BATTLEGROUNDS,2017-12-21,0,0.00,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...",Play PUBG: BATTLEGROUNDS for free. Land on str...,NaN,https://shared.akamai.steamstatic.com/store_it...,...,0,0,616738,"{'Survival': 14838, 'Shooter': 12727, 'Battle ...",59,2513842,68,16720,Survival Shooter Battle Royale Multiplayer FPS...,Play PUBG: BATTLEGROUNDS for free. Land on str...
570,Dota 2,2013-07-09,0,0.00,2,"The most-played game on Steam. Every day, mill...","The most-played game on Steam. Every day, mill...","Every day, millions of players worldwide enter...",“A modern multiplayer masterpiece.” 9.5/10 – D...,https://shared.akamai.steamstatic.com/store_it...,...,892,0,555977,"{'Free to Play': 59933, 'MOBA': 20158, 'Multip...",81,2452595,80,29366,Free to Play MOBA Multiplayer Strategy e-sport...,"Every day, millions of players worldwide enter..."
271590,Grand Theft Auto V Legacy,2015-04-13,17,0.00,0,"When a young street hustler, a retired bank ro...","When a young street hustler, a retired bank ro...",Grand Theft Auto V for PC offers players the o...,NaN,https://shared.akamai.steamstatic.com/store_it...,...,74,0,117698,"{'Open World': 32644, 'Action': 23539, 'Multip...",87,1803832,92,17517,Open World Action Multiplayer Crime Automobile...,Grand Theft Auto V for PC offers players the o...
359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,17,3.99,9,Edition Comparison Ultimate Edition The Tom Cl...,“One of the best first-person shooters ever ma...,"Tom Clancy's Rainbow Six® Siege is an elite, t...",NaN,https://shared.akamai.steamstatic.com/store_it...,...,306,80,89916,"{'FPS': 9831, 'PvP': 9162, 'e-sports': 9072, '...",84,1168020,76,12608,FPS PvP e-sports Multiplayer Tactical Shooter ...,"Tom Clancy's Rainbow Six® Siege is an elite, t..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2115110,Outrun Them,2024-02-29,0,3.99,0,The name of the game is literally the game. Ou...,The name of the game is literally the game. Ou...,It's all in the name.. outrun them.,NaN,https://shared.akamai.steamstatic.com/store_it...,...,0,0,0,"{'2D Platformer': 91, 'Runner': 85, 'Platforme...",-1,-1,-1,-1,2D Platformer Runner Platformer 2D Top-Down Ca...,It's all in the name.. outrun them. 2D Platfor...
1174200,Lands of Pharaoh: Episode 1,2020-01-07,0,10.00,0,Join Our Discord Channel About the Game Briefl...,Briefly: Lands of Pharaoh is a third person vi...,Lands of Pharaoh is a multi-camera-angle based...,NaN,https://shared.akamai.steamstatic.com/store_it...,...,0,0,0,"{'Action': 43, 'Adventure': 41, 'Indie': 41, '...",-1,-1,-1,-1,Action Adventure Indie Strategy,Lands of Pharaoh is a multi-camera-angle based...
1160190,Player One,2019-10-10,0,1.99,0,Player One is a new 3D game where you take con...,Player One is a new 3D game where you take con...,"3D Game, Thirdperson Platformer",NaN,https://shared.akamai.steamstatic.com/store_it...,...,0,0,0,"{'Indie': 31, 'Early Access': 21}",-1,-1,-1,-1,Indie Early Access,"3D Game, Thirdperson Platformer Indie Early Ac..."


In [18]:
games["tagged_description"]

appid
730        For over two decades, Counter-Strike has offer...
578080     Play PUBG: BATTLEGROUNDS for free. Land on str...
570        Every day, millions of players worldwide enter...
271590     Grand Theft Auto V for PC offers players the o...
359550     Tom Clancy's Rainbow Six® Siege is an elite, t...
                                 ...                        
2115110    It's all in the name.. outrun them. 2D Platfor...
1174200    Lands of Pharaoh is a multi-camera-angle based...
1160190    3D Game, Thirdperson Platformer Indie Early Ac...
3380340    This is an open world game on 3D, Also a missi...
763730     Ragtag Adventurers is a boss rush game that pi...
Name: tagged_description, Length: 89618, dtype: object

In [19]:
games["tagged_description"].to_csv("tagged_description.txt", index=True, header=False)

In [20]:
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

In [21]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='730,"For over two decades, Counter-Strike has offered an elite competitive experience, one shaped by millions of players from across the globe. And now the next chapter in the CS story is about to begin. This is Counter-Strike 2. FPS Shooter Multiplayer Competitive Action Team-Based e-sports Tactical First-Person PvP Online Co-Op Co-op Strategy Military War Difficult Trading Realistic Fast-Paced Moddable"\n578080,"Play PUBG: BATTLEGROUNDS for free. Land on strategic locations, loot weapons and supplies, and survive to become the last team standing across various, diverse Battlegrounds. Squad up and join the Battlegrounds for the original Battle Royale experience that only\xa0PUBG: BATTLEGROUNDS can offer. Survival Shooter Battle Royale Multiplayer FPS PvP Third-Person Shooter Action Online Co-Op Tactical Co-op First-Person Strategy Early Access Competitive Third Person Team-Based Difficult Simulation Stealth"\n570,"E

In [22]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3070


In [23]:
from sentence_transformers import SentenceTransformer
import numpy as np
import os

emb_file = "embeddings.npy"
appids_file = "appids.npy"

games = pd.read_csv("games_cleaned.csv", index_col="appid")

if os.path.exists(emb_file) and os.path.exists(appids_file):
    print("Embeddings già presenti, li carico...")
    embeddings = np.load(emb_file)
    appids = np.load(appids_file)
else:
    print("Embeddings non trovati, li calcolo...")
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device='cuda')
    texts = games["tagged_description"].tolist()
    embeddings = model.encode(texts, convert_to_numpy=True, show_progress_bar=True)
    appids = games.index.to_numpy()

    # salva gli embeddings e appid nella cartella del progetto github
    np.save(emb_file, embeddings)
    np.save(appids_file, appids)
    print("Embeddings calcolati e salvati!")

Embeddings già presenti, li carico...


In [24]:

embeddings_df = pd.DataFrame(embeddings, index=appids)
embeddings_df.columns = [f"embedding_{i}" for i in range(embeddings.shape[1])]

print(embeddings_df)

         embedding_0  embedding_1  embedding_2  embedding_3  embedding_4  \
730        -0.022541     0.024371    -0.038876    -0.040040     0.008256   
578080      0.034773    -0.039877     0.011000    -0.060675     0.038361   
570        -0.064973     0.014231     0.012559    -0.045316    -0.004826   
271590      0.025898     0.018036    -0.033346    -0.068679     0.040154   
359550     -0.031722     0.031352    -0.043098    -0.041836    -0.078538   
...              ...          ...          ...          ...          ...   
2115110     0.004244     0.001200    -0.044650    -0.050989     0.058269   
1174200     0.015897     0.051679     0.013425    -0.039386    -0.075588   
1160190    -0.029372    -0.078428    -0.025321    -0.112149    -0.037157   
3380340    -0.030730     0.015031    -0.073478    -0.094777    -0.016220   
763730      0.010343     0.036966    -0.015189    -0.012865    -0.037900   

         embedding_5  embedding_6  embedding_7  embedding_8  embedding_9  ...  \
730   

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

titles = games["name"].to_dict()

def recommend_from_multiple(appid_list, top_k=10):
    """
    Dato un elenco di appid scelti dall'utente,
    calcola il profilo utente medio e restituisce i giochi più simili.
    """
    # Trova gli indici degli appid scelti
    idxs = [np.where(appids == a)[0][0] for a in appid_list]

    # Profilo utente = media degli embedding
    user_profile = np.mean([embeddings[i] for i in idxs], axis=0)

    sims = cosine_similarity([user_profile], embeddings)[0]
    ranking = np.argsort(sims)[::-1]

    exclude = set(appid_list)
    results = [(titles[appids[i]], sims[i]) for i in ranking if appids[i] not in exclude][:top_k]

    print("Se ti sono piaciuti:")
    for a in appid_list:
        print(f"- {titles[a]}")

    print("\nTi potrebbero interessare:")
    for t, score in results:
        print(f"- {t} (similarità: {score:.2f})")

    return results

In [26]:
user_games = [appids[1], appids[4], appids[5]]

recommendations = recommend_from_multiple(user_games, top_k=5)

Se ti sono piaciuti:
- PUBG: BATTLEGROUNDS
- Tom Clancy's Rainbow Six® Siege
- Team Fortress 2

Ti potrebbero interessare:
- Trouble Town (similarità: 0.78)
- Chapter Wars (similarità: 0.78)
- Insurgency (similarità: 0.77)
- Tactical Vengeance: Play The Games (similarità: 0.77)
- Warbyte: Operation Falcon (similarità: 0.76)
